In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset

import glob
import os
import numpy as np
import pandas as pd
import random
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from tqdm import tqdm

import train_p3
from torchsummary import summary
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
src_testset = train_p3.p3(root='hw2_data/digits/usps/test',label_path='hw2_data/digits/usps/test.csv',img_size=64)
target_testset = train_p3.p3(root='hw2_data/digits/svhn/test',label_path='hw2_data/digits/svhn/test.csv',img_size=64)
min_size = 1000
data_size = range(min_size)

src_testset = Subset(src_testset, data_size)
target_trainset = Subset(target_testset,data_size)

dataset = src_testset + target_testset

dataset_loader = DataLoader(dataset, batch_size=min_size*2, shuffle=False, num_workers=4)

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model = train_p3.DANN_64x()
model_val = model.to(device)
model_val.load_state_dict(torch.load('./save_models/dann_target_svhn_improved.pth'))
model_val.eval()

print()

In [ ]:
dataiter = iter(dataset_loader)
images,labels= dataiter.next()
images = images.to(device)

FEAT = model_val(images).view(min_size*2,2048)
print(FEAT.shape)

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 10)
tsne = TSNE().fit_transform(FEAT.cpu().detach().numpy())

tx = tsne[:, 0]
ty = tsne[:, 1]
classes = np.arange(10)

sns.scatterplot(tx, ty, legend='full', hue=labels, palette=palette)

In [ ]:
source_arr = np.zeros(min_size,dtype=int)
target_arr = np.ones(min_size,dtype=int)
print(source_arr.shape,target_arr.shape)
arr = np.concatenate((source_arr,target_arr),axis=None)
print(arr.shape)
palette = sns.color_palette("bright", 2)
sns.scatterplot(tx, ty, legend='full', hue=arr, palette=palette)